In [21]:
import os
import glob
import re
import numpy as np
import pandas as pd
import scipy
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
%matplotlib inline

In [11]:
X_train_nolemma = scipy.sparse.load_npz("data/processed_descriptions/train_descriptions_noLemma.npz")
X_test_nolemma = scipy.sparse.load_npz("data/processed_descriptions/test_descriptions_noLemma.npz")
X_train_nouns_nolemma = scipy.sparse.load_npz("data/processed_descriptions/train_descriptions_nouns_noLemma.npz")
X_test_nouns_nolemma = scipy.sparse.load_npz("data/processed_descriptions/test_descriptions_nouns_noLemma.npz")

In [12]:
Y_train = scipy.sparse.load_npz("data/processed_tags/train_tag_nocat_tfdif.npz")
Y_test  = scipy.sparse.load_npz("data/processed_tags/test_tag_nocat_tfdif.npz")

In [16]:
def load_descriptions(path):
    data = []
    for filename in glob.glob(os.path.join(path, "*.txt")):
        with open(filename, "r") as description_file:
            label = os.path.splitext(os.path.basename(filename))[0]
            sentence = " ".join(line.strip() for line in description_file)
            new_row = (sentence, label) 
            data.append(new_row)

    dt = np.dtype([('sentence', object), ('label', 'int64')])
    return(np.array(data, dtype = dt))

train_descriptions_path = "data/descriptions_train/"
test_descriptions_path = "data/descriptions_test/"

train_data = load_descriptions(train_descriptions_path)
test_data = load_descriptions(test_descriptions_path)

train_data.sort(order = 'label', axis = 0)
test_data.sort(order = 'label', axis = 0)

In [2]:
def map20(test, sim_matrix): 
    total_score = 0
    for num, idx in enumerate(test):
        score = 0
        rank = np.argwhere(np.argsort(sim_matrix[num])[::-1] == num)[0][0]
        if rank <20:
            score = (20 + 1-(rank+1))/20
        else:
            score = 0 
        total_score += score
    return total_score

In [34]:
pls_resnet_all = np.loadtxt("output/nolemma_plsr_resnet_all_sims.txt")
pls_resnet_nouns = np.loadtxt("output/nolemma_plsr_resnet_noun_sims.txt")
mlp_autoencoder_tags = np.loadtxt("output/mlp_reg_sims.txt")
# lsvc_tags = np.loadtxt("output/lsvc_sims.txt")
pls_resnet_verbs = np.loadtxt("output/verbs_resnet_sims.txt")

In [31]:
merged_sims = (pls_resnet_all + pls_resnet_nouns + mlp_autoencoder_tags + pls_renset_verbs)/4

In [39]:
pls_resnet_all_ranks = (pls_resnet_all).argsort(axis = 1).argsort(axis = 1)
pls_resnet_nouns_ranks = (pls_resnet_nouns).argsort(axis = 1).argsort(axis = 1)
pls_resnet_verbs_ranks = (pls_resnet_verbs).argsort(axis = 1).argsort(axis = 1)
mlp_autoencoder_tags_ranks = (mlp_autoencoder_tags).argsort(axis = 1).argsort(axis = 1)

In [40]:
merged_ranks = (pls_resnet_all_ranks + pls_resnet_nouns_ranks + mlp_autoencoder_tags_ranks + pls_resnet_verbs_ranks)/4

In [41]:
merged_ranks

array([[ 934.  ,  420.75, 1542.25, ...,  369.  ,  332.75,  196.5 ],
       [1568.25, 1025.  ,  952.5 , ...,  383.25, 1144.  , 1888.5 ],
       [1051.25, 1024.  ,  915.  , ...,  726.75, 1693.25,  233.25],
       ...,
       [1494.75,  822.75, 1722.75, ...,  664.25,  685.5 ,  941.25],
       [1525.5 ,  339.5 , 1651.75, ...,  478.  ,  947.75,  629.  ],
       [1296.5 , 1509.  , 1813.75, ...,  838.75,  643.25,  531.75]])

In [43]:
with open('output/pls_resnet_mlp_autoencoder_v2.txt', 'w') as f:
    for i in range(0, X_test_nolemma.shape[0]):
        row = " ".join([str(img) + ".jpg" for img in np.argsort(merged_sims[i])[::-1][0:20]])
        row = str(i) + ".txt," + row
        f.write("%s\n" % row)